In [ ]:
import pandas as pd
import vectorbtpro as vbt
import numpy as np
import matplotlib.pyplot as plt
import glob



In [ ]:
vbt.settings.plotting["layout"]["template"] = "vbt_dark"
vbt.settings.plotting["layout"]["width"] = 1500
vbt.settings.plotting['layout']['height'] = 600
vbt.settings.wrapping["freq"] = "1m"
# vbt.settings.portfolio['size_granularity'] = 0.001
vbt.settings.portfolio['init_cash'] = 10000

# Wherever you saved the pickle file
spot_data_path = '/Users/ericervin/Documents/Coding/data-repository/data/BTCUSDT_1m.pkl'
futures_data_path = '/Users/ericervin/Documents/Coding/data-repository/data/BTCUSDT_1m_futures.pkl'

## Read in the LSTM recommendations

In [ ]:
# TODO: Turn this into a function

import glob
import pandas as pd
import numpy as np
# Import all of the pickle files from lstm_backtest/lstm_entries_1e5b9ae6 folder
pickle_files = glob.glob('/Users/ericervin/Documents/Coding/sigma/lstm_backtest/lstm_entries_1e5b9ae6/*.pkl')
# print(pickle_files)
data = []
for pickle_file in pickle_files:
    with open(pickle_file, 'rb') as f:
        new_file = pd.read_pickle(f)
        data.append(new_file)
data

# Define an empty list to store the concatenated dataframes
merged_data = []

# Loop over each item in the data list
for item in data:
    # Create the initial dataframe with 'Time', the prices, and 'recommendations' along with 'prediction_details'
    df = pd.DataFrame(item['price_data']['Time'], columns=['Time'])
    df[['BTCUSDT_Open', 'BTCUSDT_High', 'BTCUSDT_Low', 'BTCUSDT_Close']] = item['price_data'][['BTCUSDT_Open', 'BTCUSDT_High', 'BTCUSDT_Low', 'BTCUSDT_Close']]
    df['recommendations'] = item['recommendations']

    # Create the dataframe from 'prediction_details'
    test = pd.DataFrame(item['prediction_details'], columns=['long', 'short', 'indx_hi', 'indx_low', 'action'])

    # Reset the indexes of both dataframes
    df.reset_index(drop=True, inplace=True)
    test.reset_index(drop=True, inplace=True)

    # Concatenate the dataframes horizontally
    merged_df = pd.concat([df, test], axis=1)

    # Append the merged dataframe to the list
    merged_data.append(merged_df)

# Concatenate the dataframes in merged_data vertically
result_df = pd.concat(merged_data, axis=0)

# Reset the index of the final concatenated dataframe
result_df.reset_index(drop=True, inplace=True)

# Sort the dataframe by the 'Time' column in ascending order
result_df.sort_values('Time', inplace=True)

# Reset the index again after sorting
result_df.reset_index(drop=True, inplace=True)

# Print the final concatenated and sorted dataframe
result_df

Create the resultant forecast columns


In [ ]:
# TODO: Turn this into a function that takes in the dataframe and the number of forward prices to add
# The function will need to take in our prediction windows of 8, 18, 38, 75, 150

# Add the forward 8 prices to the dataframe
result_df['BTCUSDT_Open_1'] = result_df['BTCUSDT_Open'].shift(-1)
result_df['BTCUSDT_Open_2'] = result_df['BTCUSDT_Open'].shift(-2)
result_df['BTCUSDT_Open_3'] = result_df['BTCUSDT_Open'].shift(-3)
result_df['BTCUSDT_Open_4'] = result_df['BTCUSDT_Open'].shift(-4)
result_df['BTCUSDT_Open_5'] = result_df['BTCUSDT_Open'].shift(-5)
result_df['BTCUSDT_Open_6'] = result_df['BTCUSDT_Open'].shift(-6)
result_df['BTCUSDT_Open_7'] = result_df['BTCUSDT_Open'].shift(-7)
result_df['BTCUSDT_Open_8'] = result_df['BTCUSDT_Open'].shift(-8)


In [ ]:


# Calculate the highest high price among the forward prices
# TODO: this needs to be modified for different numbers of forward prices based on the forecast period
highest_high = result_df[['BTCUSDT_Open', 'BTCUSDT_Open_1', 'BTCUSDT_Open_2', 'BTCUSDT_Open_3', 'BTCUSDT_Open_4', 'BTCUSDT_Open_5', 'BTCUSDT_Open_6', 'BTCUSDT_Open_7', 'BTCUSDT_Open_8']].max(axis=1)
print(highest_high.shape)
# Calculate the percentage of each forward price relative to the current open price to the highest high
ranked_forward_prices = result_df[['BTCUSDT_Open', 'BTCUSDT_Open_1', 'BTCUSDT_Open_2', 'BTCUSDT_Open_3', 'BTCUSDT_Open_4', 'BTCUSDT_Open_5', 'BTCUSDT_Open_6', 'BTCUSDT_Open_7']].div(highest_high, axis=0)
print(ranked_forward_prices.shape)
# Normalize the ranked forward prices to ensure the sum is equal to 1
ranked_forward_prices_normalized = ranked_forward_prices.div(ranked_forward_prices.sum(axis=1), axis=0)
print(ranked_forward_prices_normalized.shape)
# Convert the normalized ranked forward prices to a numpy array
array = ranked_forward_prices_normalized.to_numpy()
print(array.shape)
print(array)

# Add the column as 'fwd_8_actual' in result_df
result_df['fwd_8_actual'] = array.tolist() # TODO: this may also need to be modified to handle the different forecast periods

# Print the modified result_df
result_df


## Check for accuracy
Below we measure the euclidian distance between the two arrays. 

In [ ]:
result_df['fwd_8_actual'][1]

In [ ]:
import numpy as np
result_df = result_df[:-8].copy() # TODO: this should be modified to handle the different forecast periods

# Define a function to calculate Euclidean distance
def euclidean_distance(arr1, arr2):
    return np.sqrt(np.sum((np.array(arr1) - np.array(arr2)) ** 2))

# Apply this function to the 'long' and 'fwd_8_actual' columns
result_df['long_distance_to_actual'] = result_df.apply(lambda row: euclidean_distance(row['long'], row['fwd_8_actual']), axis=1)
result_df['short_distance_to_actual'] = result_df.apply(lambda row: euclidean_distance(row['short'], row['fwd_8_actual']), axis=1)
result_df['long_minus_short'] = result_df.apply(lambda row: euclidean_distance(row['long'], row['short']), axis=1) # this is the difference between the long and short predictions

In [ ]:
result_df.index = result_df['Time']

In [ ]:
# Visualize the euclidean distance between the long predictions and the short predictions
result_df['long_minus_short'].rolling(500).mean().plot()

In [ ]:
# Assuming 'Time' column in your DataFrame 'result_df' is in datetime format and is sorted in ascending order.
# Let's calculate rolling mean, median and standard deviation of 'long_distance_to_actual' over time.

# Calculate rolling statistics over a window of, say, 500 observations.
window_size = 500
result_df['Rolling_Mean_Distance'] = result_df['long_distance_to_actual'].rolling(window=window_size).mean()
result_df['Rolling_Median_Distance'] = result_df['long_distance_to_actual'].rolling(window=window_size).median()
result_df['Rolling_Std_Dev_Distance'] = result_df['long_distance_to_actual'].rolling(window=window_size).std()
# Now calc the rolling statistics for 'short_distance_to_actual'
result_df['Rolling_Mean_Short_Distance'] = result_df['short_distance_to_actual'].rolling(window=window_size).mean()
result_df['Rolling_Median_Short_Distance'] = result_df['short_distance_to_actual'].rolling(window=window_size).median()
result_df['Rolling_Std_Dev_Short_Distance'] = result_df['short_distance_to_actual'].rolling(window=window_size).std()
# Now calc the rolling statistics for 'long_minus_short'
result_df['Rolling_Mean_Long_Minus_Short'] = result_df['long_minus_short'].rolling(window=window_size).mean()
result_df['Rolling_Median_Long_Minus_Short'] = result_df['long_minus_short'].rolling(window=window_size).median()
result_df['Rolling_Std_Dev_Long_Minus_Short'] = result_df['long_minus_short'].rolling(window=window_size).std()

# Let's plot these rolling statistics over time.
result_df[[
    'Rolling_Mean_Distance',
    # 'Rolling_Median_Distance',
    'Rolling_Std_Dev_Distance',
    'Rolling_Mean_Short_Distance',
    # 'Rolling_Median_Short_Distance',
    'Rolling_Std_Dev_Short_Distance',
    'Rolling_Mean_Long_Minus_Short',
    # 'Rolling_Median_Long_Minus_Short',
    'Rolling_Std_Dev_Long_Minus_Short'
    ]].vbt.plot(width=800).show_svg()

In [ ]:
start = '2023-01-11'
end = '2023-01-13'

result_df.loc[start:end][['long_distance_to_actual', 'short_distance_to_actual', 'long_minus_short']].vbt.plot(width=800, height=300).show_svg()
result_df.loc[start:end][['Rolling_Mean_Distance', 'Rolling_Mean_Short_Distance', 'Rolling_Mean_Long_Minus_Short']].vbt.plot(width=800, height=300).show_svg()
result_df.loc[start:end][['Rolling_Std_Dev_Distance', 'Rolling_Std_Dev_Short_Distance', 'Rolling_Std_Dev_Long_Minus_Short']].vbt.plot(width=800, height=300).show_svg()
# Plot the price
result_df.loc[start:end][['BTCUSDT_Open']].vbt.plot(width=800, height=300).show_svg()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Initialize lists to store MAE and MSE values
mae_long_fwd, mse_long_fwd = [], []
mae_short_fwd, mse_short_fwd = [], []
mae_long_short, mse_long_short = [], []

# Iterate over each row
for i in range(len(result_df)):
    long_preds = result_df['long'].iloc[i]
    short_preds = result_df['short'].iloc[i]
    actuals = result_df['fwd_8_actual'].iloc[i]

    # Calculate and store MAE and MSE values
    mae_long_fwd.append(mean_absolute_error(actuals, long_preds))
    mse_long_fwd.append(mean_squared_error(actuals, long_preds))

    mae_short_fwd.append(mean_absolute_error(actuals, short_preds))
    mse_short_fwd.append(mean_squared_error(actuals, short_preds))

    mae_long_short.append(mean_absolute_error(long_preds, short_preds))
    mse_long_short.append(mean_squared_error(long_preds, short_preds))

# Create a DataFrame to store the calculated metrics
stats_df = pd.DataFrame({
    'MAE_Long_Fwd': mae_long_fwd,
    'MSE_Long_Fwd': mse_long_fwd,
    'MAE_Short_Fwd': mae_short_fwd,
    'MSE_Short_Fwd': mse_short_fwd,
    'MAE_Long_Short': mae_long_short,
    'MSE_Long_Short': mse_long_short,
})

# Display descriptive statistics for the calculated metrics
print(stats_df.describe())


In [ ]:
# calc correlations

correlation = result_df['long_distance_to_actual'].corr(result_df['short_distance_to_actual'])

print("Correlation between Euclidean distance between long array and short array and future actual results: ", correlation)

correlation_short = result_df['long_distance_to_actual'].corr(result_df['long_minus_short'])

print("Correlation between difference in long minus short predictions and future actual results for longs: ", correlation_short)

correlation_difference = result_df['short_distance_to_actual'].corr(result_df['long_minus_short'])

print("Correlation between difference in long minus short predictions and future actual results for shorts: ", correlation_difference)

long_correlation_slope = result_df['long_slope'].corr(result_df['actual_slope'])
print(long_correlation_slope)
short_correlation_slope = result_df['short_slope'].corr(result_df['actual_slope'])
print(short_correlation_slope)

In [ ]:
# Create a dataframe to store the independent values of the 'long' and 'short' arrays and the dependent value of the 'fwd_8_actual' array
# Unpack each array into its own column and prefix each column with 'long_1, 2, 3, ...8' short_1, 2, 3, ...8 and long_minus_short_1, 2, 3, and 'y_actual_1, 2,3..8' respectively
# This is done to ensure that the column names are unique
# The 'long_minus_short' array is the difference between the 'long' and 'short' arrays

# TODO: this should be done in a loop as a function to accommodate different prediction window sizes
full_df = pd.DataFrame()
full_df['long_minus_short'] = pd.DataFrame(result_df['long_minus_short'], index=result_df.index)
full_df['long_distance_to_actual'] = pd.DataFrame(result_df['long_distance_to_actual'], index=result_df.index)
full_df['short_distance_to_actual'] = pd.DataFrame(result_df['short_distance_to_actual'], index=result_df.index)
full_df[['long_1', 'long_2', 'long_3', 'long_4', 'long_5', 'long_6', 'long_7', 'long_8']] = pd.DataFrame(result_df['long'].tolist(), index=result_df.index)
full_df[['short_1', 'short_2', 'short_3', 'short_4', 'short_5', 'short_6', 'short_7', 'short_8']] = pd.DataFrame(result_df['short'].tolist(), index=result_df.index)
full_df[['y_actual_1', 'y_actual_2', 'y_actual_3', 'y_actual_4', 'y_actual_5', 'y_actual_6', 'y_actual_7', 'y_actual_8']] = pd.DataFrame(result_df['fwd_8_actual'].tolist(), index=result_df.index)




# For sharing with the analyst team

In [ ]:
result_df.to_csv('backtest_results.csv') # This file is most of what the team will need to perform analysis
full_df.to_csv('unpacked_arrays.csv') # This is only if someone wants to get into the weeds or possibly feed into another ML Model

In [ ]:
# Create a scatterplot of the long_minus_short and long_distance_to_actual columns and then do one for short_distance_to_actual and long_minus_short
# Add the best fit line and show the rsquared value

import seaborn as sns

sns.regplot(x='long_minus_short', y='long_distance_to_actual', data=result_df)
sns.regplot(x='long_minus_short', y='short_distance_to_actual', data=result_df)
r_2 = result_df['long_minus_short'].corr(result_df['long_distance_to_actual']) ** 2
print("R-squared value for long_minus_short and long_distance_to_actual: ", r_2)
short_r_2 = result_df['long_minus_short'].corr(result_df['short_distance_to_actual']) ** 2
print("R-squared value for long_minus_short and short_distance_to_actual: ", short_r_2)


## Compare how accurate the long array is vs the short array
a heavy concentration in the lower left is what we would be hoping for

In [ ]:
sns.regplot(x='short_distance_to_actual', y='long_distance_to_actual', data=result_df)

## Let's now add the `long_minus_short` color map to see if there are any clusters

In [ ]:
from scipy import stats

long_predictions_vs_actual = result_df['long_distance_to_actual']
short_predictions_vs_actual = result_df['short_distance_to_actual']
long_minus_short = result_df['long_minus_short']

# Calculate the slope and intercept for the line of best fit
slope, intercept, r_value, p_value, std_err = stats.linregress(long_predictions_vs_actual, short_predictions_vs_actual)

plt.figure(figsize=(5, 5))
plt.scatter(long_predictions_vs_actual, short_predictions_vs_actual, c=long_minus_short, cmap='coolwarm', alpha=0.2)

# Plot the line of best fit
plt.plot(np.unique(long_predictions_vs_actual), np.poly1d(np.polyfit(long_predictions_vs_actual, short_predictions_vs_actual, 1))(np.unique(long_predictions_vs_actual)))
# Add X and Y axes labels
plt.xlabel('Long predictions vs actuals')
plt.ylabel('Short predictions vs actuals')
# Add the formula for the line of best fit
plt.text(0.2, 0.2, f'y = {slope:.2f}x + {intercept:.2f}', fontsize=15)

# Add the legend
cbar = plt.colorbar(label='Long minus short "Confidence Level" lower is better')

plt.show()


Note that when the long minus short confidence level is high (lower number) the accuracy improves significantly. (lower numbers) Notice the clustering of blue dots in the lower left quadrant

# Calculate the slopes of the arrays
I'm thinking the general direction of the arrays is an indication of a buying opportunity or selling opportunity rather than simply looking at the first element. This is just a different twist that we may want to use for signal generation

In [ ]:
# Function to calculate the slope of the weights
def calculate_slope(weights):
    t = np.arange(len(weights))
    slope, intercept = np.polyfit(t, weights, 1)
    return slope

In [ ]:
# TODO make this part of the function
result_df['long_slope'] = result_df['long'].apply(calculate_slope)
result_df['short_slope'] = result_df['short'].apply(calculate_slope)
result_df['actual_slope'] = result_df['fwd_8_actual'].apply(calculate_slope)
# format floats to 5 decimal places
pd.options.display.float_format = '{:.5f}'.format
result_df[['long_slope', 'short_slope', 'actual_slope']].describe()

In [ ]:
# Isolate the periods where the actual slope was in the top 30% of all slopes
slope_qtile = 0.99
result_df[f'actual_slope_top_{int(slope_qtile*100)}'] = result_df['actual_slope'] > result_df['actual_slope'].quantile(slope_qtile)
# Isolate the periods where the long slope was in the top 30% of all slopes
result_df[f'long_slope_top_{int(slope_qtile*100)}'] = result_df['long_slope'] > result_df['long_slope'].quantile(slope_qtile)
result_df[f'long_slope_bottom_{int(1-slope_qtile*100)}'] = result_df['long_slope'] < result_df['long_slope'].quantile(1-slope_qtile)

print(f'The number of periods where the actual slope was in the top {slope_qtile}% of all slopes: ', result_df[f'actual_slope_top_{int(slope_qtile*100)}'].sum())
print(f'The number of periods where the long slope was in the top {slope_qtile}% of all slopes: ', result_df[f'long_slope_top_{int(slope_qtile*100)}'].sum())
print(f'The average long slope during periods where the actual slope was in the top {slope_qtile}% of all slopes: ', result_df[result_df[f'actual_slope_top_{int(slope_qtile*100)}']]['long_slope'].mean())

In [ ]:
# Highlight the periods when the long minus short difference was in the bottom 30% of all differences In theory these are the most highly convicted periods
lms_qtile = 0.01
result_df[f'long_minus_short_lowest_{int(lms_qtile*100)}'] = result_df['long_minus_short'] < result_df['long_minus_short'].quantile(lms_qtile)

In [ ]:
# Create a quick backtest where the entries and exits are based on the above conditions
# If the actual slope is in the top 30% of all slopes and the long_minus_short is in the bottom 30% of all differences then go long if the long_slope and short_slope are both positive
# If the actual slope is in the top 30% of all slopes and the long_minus_short is in the bottom 30% of all differences then go short if the long_slope and short_slope are both negative
entries = pd.Series(np.where((result_df[f'long_minus_short_lowest_{int(lms_qtile*100)}'] == True) & (result_df['long_slope'] > 0) & (result_df['short_slope'] > 0), True, False))
short_entries = pd.Series(np.where((result_df[f'long_minus_short_lowest_{int(lms_qtile*100)}'] == True) & (result_df['long_slope'] < 0) & (result_df['short_slope'] < 0), True, False))

In [ ]:
# Print the number of entries and exits
print("Number of long entries: ", entries.sum())
print("Number of short entries: ", short_entries.sum())

In [ ]:
pf = vbt.Portfolio.from_signals(
    high=result_df['BTCUSDT_High'],
    low=result_df['BTCUSDT_Low'],
    open=result_df['BTCUSDT_Open'],
    close=result_df['BTCUSDT_Close'],
    entries=entries, # commented out for a short only backtest
    short_entries=short_entries,
    td_stop = 8, # Hold on to the position for 8 bars
    time_delta_format = 'Rows', # Use the row index to calculate the time delta
    # tp_stop = 0.01,
    accumulate=False,
    # sl_stop = 0.005,
    )
print(pf.stats())
# pf.plot(height=600, width=800).show_svg()
    

In [ ]:
result_df.columns

In [ ]:
# Set up entries and exits based on the slope
entry_slope_threshold = 0.07
short_entry_slope_threshold = 0.08
entries = pd.Series(np.where((result_df['long_slope'] > entry_slope_threshold), True, False))
short_entries = pd.Series(np.where((result_df['short_slope'] < -short_entry_slope_threshold), True, False))
prediction_period = 8
pf = vbt.Portfolio.from_signals(
    high=result_df['BTCUSDT_High'],
    low=result_df['BTCUSDT_Low'],
    open=result_df['BTCUSDT_Open'],
    close=result_df['BTCUSDT_Close'],
    entries=entries, # commented out for a short only backtest
    # exits = result_df['long_slope']<0.04,
    short_entries=short_entries,
    td_stop = prediction_period, # Hold on to the position for 8 bars
    time_delta_format = 'Rows', # Use the row index to calculate the time delta
    # tp_stop = 0.01,
    accumulate=False,
    # sl_stop = 0.005,
    )
print(pf.stats())
# pf.plot(height=600, width=800).show()

# Create a filtered dataset
pick a threshold that you will only accept recommendations when the long_minus_short is below XX. I'm going to start with 0.4

In [ ]:
result_df[result_df['long_minus_short'] < 0.1][['long_minus_short','long_distance_to_actual','short_distance_to_actual','long_slope', 'short_slope', 'actual_slope']]

# Let's see how we would do if we listened to the LSTM

import the objects action types. 

In [ ]:
from enum import Enum


class ActionType(str, Enum):
    OPEN_SHORT = "open-short"
    OPEN_LONG = "open-long"
    CLOSE_LONG = "close-long"
    CLOSE_SHORT = "close-short"
    LEGACY_OPEN_LONG = "open_long"
    LEGACY_OPEN_SHORT = "open_short"
    LEGACY_CLOSE_LONG = "close_long"
    LEGACY_CLOSE_SHORT = "close_short"
    NOOP = "no_op"


def get_close_action(direction: ActionType):
    if direction == ActionType.OPEN_LONG:
        return ActionType.CLOSE_LONG
    elif direction == ActionType.OPEN_SHORT:
        return ActionType.CLOSE_SHORT

In [ ]:
# Create entries and exits based on the long and short predictions but filtered for low values of long_minus_short
lms_filter = 0.1
entries_lms         = pd.Series(np.where((result_df['long_minus_short'] < lms_filter) & (result_df['recommendations']==ActionType.OPEN_LONG), True, False))
short_entries_lms   = pd.Series(np.where((result_df['long_minus_short'] < lms_filter) & (result_df['recommendations']==ActionType.OPEN_SHORT), True, False))
print(entries.sum())
print(short_entries.sum())
long_slope_filter = 0.02
entries_slope         = pd.Series(np.where((result_df['long_slope'] > long_slope_filter) & (result_df['recommendations']==ActionType.OPEN_LONG), True, False))
short_entries_slope   = pd.Series(np.where((result_df['short_slope'] < -long_slope_filter) & (result_df['recommendations']==ActionType.OPEN_SHORT), True, False))
print(entries_slope.sum())
print(short_entries_slope.sum())

In [ ]:


pf = vbt.Portfolio.from_signals(
    high=result_df['BTCUSDT_High'],
    low=result_df['BTCUSDT_Low'],
    open=result_df['BTCUSDT_Open'],
    close=result_df['BTCUSDT_Close'], 
    # entries=entries_slope, # comment this out for a short only portfolio simulation
    # short_entries=short_entries_slope,
    entries=entries_lms, # comment this out for a short only portfolio simulation
    short_entries=short_entries_lms,
    td_stop = 8,# Note we are only holding for 8 bars
    time_delta_format = 'Rows',
    # tp_stop = 0.01,
    # accumulate=False,
    # sl_stop = 0.005,
    leverage=2,
    )
print(pf.stats())
pf.plot(height=600, width=800).show_svg()


In [ ]:
# Clean the signals
clean_entries, clean_exits = entries.vbt.signals.clean(short_entries)
clean_short_entries, clean_short_exits = short_entries.vbt.signals.clean(entries)
print(clean_entries.sum())
print(clean_short_entries.sum())

In [ ]:
clean_pf = vbt.Portfolio.from_signals(
    high=result_df['BTCUSDT_High'],
    low=result_df['BTCUSDT_Low'],
    open=result_df['BTCUSDT_Open'],
    close=result_df['BTCUSDT_Close'],
    entries=clean_entries,
    exits=clean_exits,
    short_entries=clean_short_entries,
    short_exits=clean_short_exits,
    # td_stop = 8,
    # time_delta_format = 'Rows',
    # tp_stop = 0.01,
    accumulate=False,
    # sl_stop = 0.05,
    )
print(clean_pf.stats())
clean_pf.plot(height=600, width=800).show_svg()


In [ ]:


def dollar_bar_func(ohlc_df, dollar_bar_size):
    # Calculate dollar value traded for each row
    ohlc_df['DollarValue'] = ohlc_df['Close'] * ohlc_df['Volume']
    
    # Calculate cumulative dollar value
    ohlc_df['CumulativeDollarValue'] = ohlc_df['DollarValue'].cumsum()
    
    # Determine the number of dollar bars
    num_bars = int(ohlc_df['CumulativeDollarValue'].iloc[-1] / dollar_bar_size)
    
    # Generate index positions for dollar bars
    bar_indices = [0]
    cumulative_value = 0
    for i in range(1, len(ohlc_df)):
        cumulative_value += ohlc_df['DollarValue'].iloc[i]
        if cumulative_value >= dollar_bar_size:
            bar_indices.append(i)
            cumulative_value = 0
    
    # Create a new dataframe with dollar bars
    dollar_bars = []
    for i in range(len(bar_indices) - 1):
        start_idx = bar_indices[i]
        end_idx = bar_indices[i + 1]
        # TODO: allow the original dataframe to be dynamic to retain all of the original columns and data so the user can pass a dictionary for how to aggregate each column
        dollar_bar = {
            'Open': ohlc_df['Open'].iloc[start_idx],
            'High': ohlc_df['High'].iloc[start_idx:end_idx].max(),
            'Low': ohlc_df['Low'].iloc[start_idx:end_idx].min(),
            'Close': ohlc_df['Close'].iloc[end_idx],
            'Volume': ohlc_df['Volume'].iloc[start_idx:end_idx].sum(),
            'Quote volume': ohlc_df['Quote volume'].iloc[start_idx:end_idx].sum(),
            'Trade count': ohlc_df['Trade count'].iloc[start_idx:end_idx].sum(),
            'Taker base volume': ohlc_df['Taker base volume'].iloc[start_idx:end_idx].sum(),
            'Taker quote volume': ohlc_df['Taker quote volume'].iloc[start_idx:end_idx].sum()
        }
        
        if isinstance(ohlc_df.index, pd.DatetimeIndex):
            dollar_bar['Open Time'] = ohlc_df.index[start_idx]
            dollar_bar['Close Time'] = ohlc_df.index[end_idx] - pd.Timedelta(milliseconds=1)
        elif 'Open Time' in ohlc_df.columns:
            dollar_bar['Open Time'] = ohlc_df['Open Time'].iloc[start_idx]
            dollar_bar['Close Time'] = ohlc_df['Open Time'].iloc[end_idx] - pd.Timedelta(milliseconds=1)
        
        dollar_bars.append(dollar_bar)
    
    dollar_bars_df = pd.concat([pd.DataFrame([bar]) for bar in dollar_bars], ignore_index=True)
    # Set the index to be the Open Time
    dollar_bars_df.set_index('Open Time', inplace=True)
    return dollar_bars_df

# Create a simple function to simplify the number so we can use it in our column names
def simplify_number(num):
    """
    Simplifies a large number by converting it to a shorter representation with a suffix (K, M, B).
    simplify_number(1000) -> 1K
    """
    suffixes = ['', 'K', 'M', 'B']
    suffix_index = 0
    while abs(num) >= 1000 and suffix_index < len(suffixes) - 1:
        num /= 1000.0
        suffix_index += 1
    suffix = suffixes[suffix_index] if suffix_index > 0 else ''
    simplified_num = f'{int(num)}{suffix}'
    return simplified_num

def custom_merge_and_fill_dollar_bars(original_df, dollar_bars_df, dollar_bar_size):
    """
    Merges the original dataframe with the dollar bars dataframe and fills the NaN values.
    """
    # Add prefix to column names in dollar bars dataframe
    # Reset the index
    dollar_bars_df.reset_index(inplace=True) # in case open time is the index, we need to reset it to a column so we can rename it  
    dollar_bar_prefix = f'db_{simplify_number(dollar_bar_size)}_' # prefix for dollar bar columns eg 'db_90M_Open Time'
    dollar_bars_df_renamed = dollar_bars_df.add_prefix(dollar_bar_prefix)

    # Convert 'Open Time' columns to pandas datetime format and set them as index
    dollar_bars_df_renamed.index = pd.to_datetime(dollar_bars_df_renamed[dollar_bar_prefix + 'Open Time']) # TODO make this dynamic

    # Merge the dataframes on the index
    merged_df = original_df.merge(dollar_bars_df_renamed, how='left', left_index=True, right_index=True)

    # Set the flag for a new dollar bar with prefix
    merged_df[dollar_bar_prefix + 'NewDBFlag'] = ~merged_df[dollar_bar_prefix + 'BTCUSDT_Close'].isna() # TODO: make this dynamic

    # Forward fill the NaN values for all columns except the new dollar bar flag
    columns_to_ffill = [col for col in merged_df.columns if col != dollar_bar_prefix + 'NewDBFlag']
    merged_df[columns_to_ffill] = merged_df[columns_to_ffill].fillna(method='ffill')

    # Fill the remaining NaN values in the new dollar bar flag column with False
    merged_df[dollar_bar_prefix + 'NewDBFlag'] = merged_df[dollar_bar_prefix + 'NewDBFlag'].fillna(False)
    
    # Assign the renamed 'Open Time' column back to the dataframe
    merged_df[dollar_bar_prefix + 'Open Time'] = merged_df[dollar_bar_prefix + 'Open Time'] # TODO: make this dynamic 
    return merged_df


In [ ]:
def custom_merge_and_fill_dollar_bars(original_df, dollar_bars_df, dollar_bar_size):
    """
    Merges the original dataframe with the dollar bars dataframe and fills the NaN values.
    """
    # Add prefix to column names in dollar bars dataframe
    dollar_bar_prefix = f'db_{simplify_number(dollar_bar_size)}_' # prefix for dollar bar columns eg 'db_90M_Open Time'
    
    # Create a copy of the dollar bars dataframe and reset its index
    dollar_bars_df = dollar_bars_df.reset_index().copy()  

    # Add prefix to columns
    dollar_bars_df_renamed = dollar_bars_df.add_prefix(dollar_bar_prefix)

    # Convert 'Open Time' columns to pandas datetime format and set them as index
    dollar_bars_df_renamed.index = pd.to_datetime(dollar_bars_df_renamed[dollar_bar_prefix + 'Open Time']) 

    # Create a new dataframe with the same index as the original dataframe and fill it with the values from the dollar bars dataframe
    new_dollar_bars_df = pd.DataFrame(index=original_df.index)
    new_dollar_bars_df = new_dollar_bars_df.merge(dollar_bars_df_renamed, how='left', left_index=True, right_index=True)

    # Merge the original dataframe with the new dollar bars dataframe
    merged_df = original_df.merge(new_dollar_bars_df, how='left', left_index=True, right_index=True)

    # Set the flag for a new dollar bar with prefix
    merged_df[dollar_bar_prefix + 'NewDBFlag'] = ~merged_df[dollar_bar_prefix + 'BTCUSDT_Close'].isna()

    # Forward fill the NaN values for all columns except the new dollar bar flag
    columns_to_ffill = [col for col in merged_df.columns if col != dollar_bar_prefix + 'NewDBFlag']
    merged_df[columns_to_ffill] = merged_df[columns_to_ffill].fillna(method='ffill')

    # Fill the remaining NaN values in the new dollar bar flag column with False
    merged_df[dollar_bar_prefix + 'NewDBFlag'] = merged_df[dollar_bar_prefix + 'NewDBFlag'].fillna(False)
    
    return merged_df


In [ ]:
dollar_bar_size = 90_000_000 # 90 million
dollar_bars_df = result_df.copy()
futures_1m = vbt.BinanceData.load(futures_data_path).get()
original_df = futures_1m.copy()

dollar_bar_prefix = f'db_{simplify_number(dollar_bar_size)}_' # prefix for dollar bar columns eg 'db_90M_Open Time'
# Create a copy of the dollar bars dataframe and reset its index
# dollar_bars_df = dollar_bars_df.reset_index().copy()  
# Rename the column Time to Open Time
dollar_bars_df.rename(columns={'Time': 'Open Time'}, inplace=True)
# Add prefix to columns
dollar_bars_df_renamed = dollar_bars_df.add_prefix(dollar_bar_prefix)
dollar_bars_df_renamed

In [ ]:
# Merge this with the 1 minute futures data so stop losses and take profits can be calculated more appropriately
copy_result_df = result_df.copy()
print(copy_result_df.columns)
# Rename the index to Open Time
copy_result_df.index.rename('Open Time', inplace=True)
# copy_result_df.reset_index(inplace=True)
print(copy_result_df.columns)
copy_result_df.index = pd.to_datetime(copy_result_df.index)
futures_1m.index = pd.to_datetime(futures_1m.index)

full_1m_df = custom_merge_and_fill_dollar_bars(futures_1m,copy_result_df, dollar_bar_size=90_000_000,)
# full_1m_df[['Close', 'db_90M_Open Time', 'db_90M_BTCUSDT_Close']]



In [ ]:
# Remove the first 8 days and remove everything after '2023-03-22' the end of the result_df
full_1m_df = full_1m_df.loc['2021-01-09': '2023-03-22']

In [ ]:
full_1m_df['db_90M_NewDBFlag'].sum() # How many dollar bars were created

In [ ]:
# Plot the dollar bar close with the 1 minute close to see if they match
full_1m_df.loc['2023-01-01':'2023-01-03'][['Close', 'db_90M_BTCUSDT_Close']].vbt.plot(title='BTCUSDT Close vs 90M BTCUSDT Close')